In [2]:
import json
import pandas as pd

with open('../data/annotation_data/longform_eval_first_3_samples_paragraph_annotations (2).json') as f:
    pretest = json.load(f)

In [3]:
raters = {

}
rating_to_number = {
    'supports': 0,
    'neutral': 1,
    'contradicts': 2
}
agreements = {
    1: 0,
    2: 0,
    3: 0,
    4: 0
}
agreements_by_class = {
    'supports': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'neutral': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'contradicts': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    }
}
annos = []
for example in pretest['examples']:
    agreements[
        len(example['classifications'])
    ] += 1
    for rating in example['classifications']:
        for rater in rating['classified_by']:
            anon_id = rater['annotator_id']
            if anon_id not in raters:
                raters[anon_id] = []
            raters[anon_id].append(rating_to_number[rating['classname']])
            agreements_by_class[rating['classname']][len(example['classifications'])] += 1
            annos.append({
                'anno_id': anon_id,
                'sample_id':example['metadata']['sample'],
                'example_id': example['example_id'],
                'rating': rating['classname'],
                'rating_num': rating_to_number[rating['classname']],
            })
anno_df = pd.DataFrame(annos)
len(anno_df)

425

In [4]:
# get example_id: rating for each annotator
two_samples = []
three_samples = []
four_samples = []
for example_id in anno_df.example_id.unique():
    sample_ratings = anno_df[anno_df.example_id == example_id]
    if len(sample_ratings) == 2:
        two_samples.append(sample_ratings['rating_num'].values)
    elif len(sample_ratings) == 3:
        three_samples.append(sample_ratings['rating_num'].values)
        two_samples.append(sample_ratings['rating_num'].values[:2])
        two_samples.append(sample_ratings['rating_num'].values[1:3])
    elif len(sample_ratings) == 4:
        four_samples.append(sample_ratings['rating_num'].values)

In [5]:
len(two_samples), len(three_samples), len(four_samples)

(239, 54, 0)

In [6]:
import krippendorff as kd
import numpy as np
kd.alpha(
    np.array(three_samples).T,
)

0.5683305732986358

In [7]:
# cohens kappa
from sklearn.metrics import cohen_kappa_score

print("Three samples, 1 and 2", cohen_kappa_score(
    np.array(three_samples).T[1],
    np.array(three_samples).T[2]
))
print("Three samples, 0 and 1", cohen_kappa_score(
    np.array(three_samples).T[0],
    np.array(three_samples).T[1]
))
print("Three samples, 0 and 2", cohen_kappa_score(
    np.array(three_samples).T[0],
    np.array(three_samples).T[2]
))
print("Two samples, 0 and 1", cohen_kappa_score(
    np.array(two_samples).T[0],
    np.array(two_samples).T[1]
))

Three samples, 1 and 2 0.7339901477832512
Three samples, 0 and 1 0.6324446965399886
Three samples, 0 and 2 0.43925233644859807
Two samples, 0 and 1 0.6462365001918755


In [16]:
len(np.array(three_samples).T[0])

54

In [24]:
from statsmodels.stats import inter_rater as irr
agg = irr.aggregate_raters(
    np.array(two_samples) 
)
irr.fleiss_kappa(agg[0], method='fleiss')

0.6382441977800202

In [136]:
import json
import pandas as pd

gpt_35_annotations_human_path = '../results/annotations_llama2_chat_human_edit_pretest_annotation_gpt-3.5-turbo-0613.json'
gpt_35_annotations_human = json.load(open(gpt_35_annotations_human_path))

gpt_35_annotations_no_edit_path = '../results/annotations_llama2_chat_no_edit_pretest_annotation_gpt-3.5-turbo-0613.json'
gpt_35_annotations_no_edit = json.load(open(gpt_35_annotations_no_edit_path))

gpt_35_annotations_edit_path = '../results/annotations_llama2_chat_rome_edit_pretest_annotation_gpt-3.5-turbo-0613.json'
gpt_35_annotations_edit = json.load(open(gpt_35_annotations_edit_path))

gpt_35_df = pd.DataFrame(
    gpt_35_annotations_human + gpt_35_annotations_no_edit + gpt_35_annotations_edit
)

gpt_4_annotations_human_path = '../results/annotations_llama2_chat_human_edit_pretest_annotation_gpt-4.json'
gpt_4_annotations_human = json.load(open(gpt_4_annotations_human_path))

gpt_4_annotations_no_edit_path = '../results/annotations_llama2_chat_no_edit_pretest_annotation_gpt-4.json'
gpt_4_annotations_no_edit = json.load(open(gpt_4_annotations_no_edit_path))

gpt_4_annotations_edit_path = '../results/annotations_llama2_chat_rome_edit_pretest_annotation_gpt-4.json'
gpt_4_annotations_edit = json.load(open(gpt_4_annotations_edit_path))

gpt_4_df = pd.DataFrame(
    gpt_4_annotations_human + gpt_4_annotations_no_edit + gpt_4_annotations_edit
)





In [146]:
import json

with open('../data/annotation_data/longform_eval_first_3_samples_paragraph_annotations.json') as f:
    pretest = json.load(f)

intervention_map = {
    'human': 'llama2_chat_human_edit_pretest_annotation',
    'no_edit': 'llama2_chat_no_edit_pretest_annotation',
    'rome': 'llama2_chat_rome_edit_pretest_annotation'
}

raters = {
    9: [],
    10: []
}
rating_to_number = {
    'supports': 0,
    'neutral': 1,
    'contradicts': 2
}
agreements = {
    1: 0,
    2: 0,
    3: 0,
    4: 0
}
agreements_by_class = {
    'contradicts': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'neutral': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'supports': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    }
}
for example in pretest['examples']:
    agreements[
        len(example['classifications'])
    ] += 1
    for rating in example['classifications']:
        for rater in rating['classified_by']:
            anon_id = rater['annotator_id']
            if anon_id not in raters:
                raters[anon_id] = []
            
            agreements_by_class[rating['classname']][len(example['classifications'])] += 1
            raters[9].append(
                gpt_35_df.loc[
                    (gpt_35_df['content'] == example['content'])
                ]['classification'].values[0]
            )
            raters[10].append(
                gpt_4_df.loc[
                    (gpt_4_df['content'] == example['content'])
                ]['classification'].values[0]
            )
            raters[anon_id].append(rating['classname'])
            

In [155]:
# cohens kappa

from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(
    raters[10],
    raters[5]
)

KeyError: 5